In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!mkdir volume_maps/
!mkdir stiffness_maps/
!mkdir splits/
!cp -r drive/Shareddrives/'Brain Voxels'/'Final Data Used for Paper'/Volume_FINAL/. volume_maps/
!cp -r drive/Shareddrives/'Brain Voxels'/'Final Data Used for Paper'/Stiffness_FINAL/. stiffness_maps/
!cp -r drive/Shareddrives/'Brain Voxels'/splits/. splits/
!cp -r drive/Shareddrives/'Brain Voxels'/labels_final.csv .

In [5]:
!pip install keras-hypetune

In [6]:
import nibabel as nib
import tensorflow as tf
from tensorflow.keras import layers
import pathlib
from scipy.interpolate import RegularGridInterpolator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from keras.utils.vis_utils import plot_model
from sklearn.metrics import mean_absolute_error as mae
from numpy.random import seed
import os
import pandas as pd
import sklearn
import sklearn.model_selection
import numpy as np
import matplotlib.pyplot as plt
import random
from kerashypetune import KerasRandomSearchCV

In [7]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [45]:
train_id = pd.read_csv('splits/train_split.csv', delimiter=',', header=None).to_numpy().squeeze()
val_id = pd.read_csv('splits/val_split.csv', delimiter=',', header=None).to_numpy().squeeze()
test_id = pd.read_csv('splits/test_split.csv', delimiter=',', header=None).to_numpy().squeeze()

In [9]:
def common_path(arr, pos='prefix'):
    # The longest common prefix of an empty array is "".
    if not arr:
        print("Longest common", pos, ":", "")
    # The longest common prefix of an array containing 
    # only one element is that element itself.
    elif len(arr) == 1:
        print("Longest common", pos, ":", str(arr[0]))
    else:
        dir = range(len(arr[0])) if pos=="prefix" else range(-1,-len(arr[0])+1,-1)
        # Sort the array
        arr.sort()
        result = ""
        # Compare the first and the last string character
        # by character.
        for i in dir:
            #  If the characters match, append the character to
            #  the result.
            if arr[0][i] == arr[-1][i]:
                result += arr[0][i]
            # Else, stop the comparison
            else:
                break
    if pos=="suffix":
        result = result[::-1]
    print("Longest common", pos, ":", result)
    return result

In [43]:
def read_files(data_folder_path, label_folder_path, set_id, only_map=False):
    labels = pd.read_csv(label_folder_path+'labels_final.csv')
    labels_list = []
    map_list = []
    sex_list = []
    study_list = []
    meta_list = []
    for root, dirs, files in os.walk(data_folder_path):
        common_prefix = common_path(files, pos="prefix")
        common_suffix = common_path(files, pos="suffix")
        for id in set_id:
            age =  labels.loc[labels["ID"] == id,'Age'].to_numpy()[0]
            sex =  labels.loc[labels["ID"] == id,'Antipodal_Sex'].to_numpy()[0]
            study = labels.loc[labels["ID"] == id,'Study_ID'].to_numpy()[0]
            filename = common_prefix + str(id) + common_suffix
            try:
                nib_raw = nib.load(data_folder_path + filename)
            except FileNotFoundError:
                filename = common_prefix + "{:0>3d}".format(id) + common_suffix
                try:
                    nib_raw = nib.load(data_folder_path + filename)
                except FileNotFoundError:
                    print(id)
                    continue
            meta = nib_raw.header
            map = nib_raw.get_fdata()[:,:,:]
            labels_list.append(age)
            sex_list.append(sex)
            map_list.append(map)
            study_list.append(study)
            meta_list.append(meta)
    X_map = np.array(map_list).astype(np.float32)
    X_sex = np.array(sex_list)
    X_study = np.array(study_list)
    y = np.array(labels_list).astype(np.float32)
    m = np.array(meta_list)
    if only_map:
        output = X_map
    else:
        output = (X_map, X_sex, X_study, y, m)
    return output

In [11]:
def preprocess(X_train, X_test, preproc_type='std'):
    X_train_pp, X_test_pp = np.zeros_like(X_train), np.zeros_like(X_test)
    if preproc_type == 'scaling':
        X_train_pp = np.nan_to_num(X_train,nan=-750)/10000
        X_test_pp = np.nan_to_num(X_test,nan=-750)/10000

    elif preproc_type == 'std':
        mu = np.nanmean(X_train)
        sigma = np.nanstd(X_train)
        X_train_pp = (X_train-mu)/sigma
        mu_adj = np.nanmean(X_train_pp)
        X_train_pp[np.isnan(X_train_pp)] = mu_adj

        X_test_pp = (X_test-mu)/sigma
        X_test_pp[np.isnan(X_test_pp)] = mu_adj

    elif preproc_type == 'max-min':
        delta = np.nanmax(X_train)-np.nanmin(X_train)
        min = np.nanmin(X_train)
        X_train_pp = (X_train-min)/delta
        mu_adj = np.nanmin(X_train_pp) #?
        X_train_pp[np.isnan(X_train_pp)] = mu_adj

        X_test_pp = (X_test-min)/delta
        X_test_pp[np.isnan(X_test_pp)] = mu_adj

    return X_train_pp, X_test_pp


In [ ]:
def make_model(param):
    # Add categorical input type 
    if param['cat_input_type'] == 'None':
        condition = lambda x: x*np.zeros(10)
        # condition = lambda x: x*0
    elif param['cat_input_type'] == 'sex':
        condition = lambda x: x*np.concatenate((np.ones(2),np.zeros(8)))
        # condition = lambda x: x[:,:2]
    elif param['cat_input_type'] == 'study':
        condition = lambda x: x*np.concatenate((np.zeros(2),np.ones(8)))
        # condition = lambda x: x[:,2:]
    elif param['cat_input_type'] == 'sex_study':
        # mask = np.ones(10)
        # condition = lambda x: x
        condition = lambda x: x
    else:
        raise Exception('Categorical input type selected is undefined')

    initializer = lambda n_chan: tf.keras.initializers.he_normal()
    n_chan = 8

    # n-Maps definition
    map_models = dict()
    models_outputs = []
    models_inputs = []
    for k in range(param['n_maps']):
        map_models[str(k)] = tf.keras.Sequential()
        map_models[str(k)].add(tf.keras.Input(shape=(91,109,55)))
        map_models[str(k)].add(layers.Reshape((91,109,55,1)))
        models_inputs.append(map_models[str(k)].input)
        models_outputs.append(map_models[str(k)].output)

    x = tf.keras.layers.concatenate(models_outputs)

    if param['arc_type'] == 1:
        map_shape = x.get_shape()
        model_cat = tf.keras.Sequential()
        model_cat.add(tf.keras.Input(shape=10))
        model_cat.add(layers.Lambda(condition))
        model_cat.add(layers.Dense(np.prod(map_shape[1:])))
        model_cat.add(layers.Reshape(map_shape[1:]))
        x = layers.Add()([x,model_cat.output])
    
    for layer in range(5):
        x = layers.Conv3DTranspose(2**layer*n_chan, (3, 3, 3), strides=(1, 1, 1), padding='same', kernel_initializer=initializer(n_chan), use_bias=False)(x)
        x = layers.ReLU()(x)
        x = layers.Conv3DTranspose(2**layer*n_chan, (3, 3, 3), strides=(1, 1, 1), padding='same', kernel_initializer=initializer(n_chan), use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(x)
    
    concat_layer_shape = x.get_shape()# model.layers
    if param['arc_type'] == 1:
        x = layers.Reshape((np.prod(concat_layer_shape[1:4])*(concat_layer_shape[-1]),))(x)
        x = layers.Dense(640, activation="relu")(x) # try activation="softplus"
        x = layers.Dense(100, activation="relu")(x) # try activation="softplus"
    elif param['arc_type'] == 2:
        x = layers.Reshape((np.prod(concat_layer_shape[1:4])*(concat_layer_shape[-1]),))(x)
        model = tf.keras.models.Model(inputs=models_inputs, outputs=x)

        model_cat = tf.keras.Sequential()
        model_cat.add(tf.keras.Input(shape=10))
        model_cat.add(layers.Lambda(condition))

        y = layers.concatenate([model.output, model_cat.output])
        x = layers.Dense(640, activation="relu")(y) # try activation="softplus"
        x = layers.Dense(100, activation="relu")(x) # try activation="softplus"
    elif param['arc_type'] == 3:
        x = layers.Reshape((np.prod(concat_layer_shape[1:4])*(concat_layer_shape[-1]),))(x)
        x = layers.Dense(640, activation="relu")(x) # try activation="softplus"
        model = tf.keras.models.Model(inputs=models_inputs, outputs=x)

        model_cat = tf.keras.Sequential()
        model_cat.add(tf.keras.Input(shape=10))
        model_cat.add(layers.Lambda(condition))

        y = layers.concatenate([model.output, model_cat.output])
        x = layers.Dense(100, activation="relu")(y) # try activation="softplus"
    elif param['arc_type'] == 4:
        x = layers.Reshape((np.prod(concat_layer_shape[1:4])*(concat_layer_shape[-1]),))(x)
        x = layers.Dense(640, activation="relu")(x) # try activation="softplus"
        x = layers.Dense(100, activation="relu")(x) # try activation="softplus"
        model = tf.keras.models.Model(inputs=models_inputs, outputs=x)

        model_cat = tf.keras.Sequential()
        model_cat.add(tf.keras.Input(shape=10))
        model_cat.add(layers.Lambda(condition))

        x = tf.keras.layers.concatenate([model.output, model_cat.output])
    else:
        raise Exception('Architecture type selected is undefined')

    x = layers.Dense(1, activation="linear")(x)
    models_inputs.append(model_cat.input) 
    final_model = tf.keras.models.Model(inputs=models_inputs, outputs=x)
    final_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=param['lr'], beta_1=0.9, 
                                        beta_2=0.999, epsilon=1e-07, amsgrad=False,
                                        name='Adam'), 
    loss = tf.keras.losses.MeanAbsoluteError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError(), 
            tf.keras.metrics.MeanAbsoluteError()])
    return final_model

param = dict()
param['cat_input_type'] = 'sex'
param['arc_type'] = 1
param['lr'] = 5e-5
param['n_maps'] = 2
model=make_model(param)
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [47]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
folder_path_input_volume = 'volume_maps/'
folder_path_input_stiffness = 'stiffness_maps/'
folder_path_labels = ''

print('Loading stiffness maps training instances')
X_train_stf, X_train_sex, X_train_study, y_train, m_train = read_files(folder_path_input_stiffness, folder_path_labels, train_id)
print('Loading stiffness maps validation instances')
X_val_stf, X_val_sex, X_val_study, y_val, m_val = read_files(folder_path_input_stiffness, folder_path_labels, val_id)
print('Loading stiffness maps test instances')
X_test_stf, X_test_sex, X_test_study, y_test, m_test = read_files(folder_path_input_stiffness, folder_path_labels, test_id)

print('Loading volume maps training instances')
X_train_vol = read_files(folder_path_input_volume, folder_path_labels, train_id, only_map=True)
print('Loading volume maps validation instances')
X_val_vol = read_files(folder_path_input_volume, folder_path_labels, val_id, only_map=True)
print('Loading volume maps test instances')
X_test_vol = read_files(folder_path_input_volume, folder_path_labels, test_id, only_map=True)

# One hot encoding for categorical variables
# define one hot encoding
encoder = OneHotEncoder(sparse=False)
# transform categorical variables
X_train_sex = encoder.fit_transform(X_train_sex.reshape(-1,1))
X_val_sex = encoder.fit_transform(X_val_sex.reshape(-1,1))
X_test_sex = encoder.fit_transform(X_test_sex.reshape(-1,1))
X_train_study = encoder.fit_transform(X_train_study.reshape(-1,1))
X_val_study = encoder.fit_transform(X_val_study.reshape(-1,1))
X_test_study = encoder.fit_transform(X_test_study.reshape(-1,1))

# Merge train and validation sets
X_train_stf = np.concatenate((X_train_stf, X_val_stf), axis=0)
X_train_vol = np.concatenate((X_train_vol, X_val_vol), axis=0)
X_train_sex = np.concatenate((X_train_sex, X_val_sex), axis=0)
X_train_study = np.concatenate((X_train_study, X_val_study), axis=0)
y_train = np.concatenate((y_train, y_val), axis=0)

Loading stiffness maps training instances
Longest common prefix : Stiffness_
Longest common suffix : .nii
Loading stiffness maps validation instances
Longest common prefix : Stiffness_
Longest common suffix : .nii
Loading stiffness maps test instances
Longest common prefix : Stiffness_
Longest common suffix : .nii
Loading volume maps training instances
Longest common prefix : MPRAGE_
Longest common suffix : _struc_GM_to_T.nii
Loading volume maps validation instances
Longest common prefix : MPRAGE_
Longest common suffix : _struc_GM_to_T.nii
Loading volume maps test instances
Longest common prefix : MPRAGE_
Longest common suffix : _struc_GM_to_T.nii


In [48]:
seed = 12345
PREPROC_TYPE = 'std'
X_train_stf_pp, X_test_stf_pp = preprocess(X_train_stf, X_test_stf, preproc_type=PREPROC_TYPE)
X_train_vol_pp, X_test_vol_pp = preprocess(X_train_vol, X_test_vol, preproc_type=PREPROC_TYPE)
# Hyperparameter Grid
param_grid = {
    # 'arc_type' : [1],
    'arc_type' : [1, 2, 3, 4],
    # 'lr' : [5e-5],
    # 'lr' : stats.uniform(1e-4, 0.1),
    'lr' : [1e-2, 1e-3, 1e-4, 1e-5],
    # 'batch_size' : [4],
    'batch_size' : [4, 12, 20, 28],
    'epochs' : [40],
    # 'epochs' : [20, 30, 40],
    'cat_input_type': ['None', 'sex', 'study', 'sex_study'],
    # 'cat_input_type': ['sex_study']
    'n_maps' : [2]
}
# Define model
# wrap our model into a scikit-learn compatible classifier
print("[INFO] initializing model...")
seed_everything(seed)
cv = KFold(n_splits=5, random_state=seed, shuffle=True)
krs = KerasRandomSearchCV(make_model, param_grid, cv=cv, monitor='val_loss', greater_is_better=False,
                          n_iter=1, sampling_seed=seed)


[INFO] initializing model...


In [49]:
X_train_cat = np.concatenate((X_train_sex,X_train_study), axis=1)
print("[INFO] performing random search...")
seed_everything(seed)
trainData = [X_train_stf_pp, X_train_vol_pp, X_train_cat]
trainTarget = y_train
krs.search(trainData, trainTarget)

[INFO] performing random search...

##################
###  Fold 001  ###
##################

1 trials detected for ('arc_type', 'lr', 'batch_size', 'epochs', 'cat_input_type', 'n_maps')

***** (1/1) *****
Search({'arc_type': 4, 'lr': 0.01, 'batch_size': 12, 'epochs': 40, 'cat_input_type': 'sex_study', 'n_maps': 2})
SCORE: 86.00097 at epoch 27

##################
###  Fold 002  ###
##################

1 trials detected for ('arc_type', 'lr', 'batch_size', 'epochs', 'cat_input_type', 'n_maps')

***** (1/1) *****
Search({'arc_type': 4, 'lr': 0.01, 'batch_size': 12, 'epochs': 40, 'cat_input_type': 'sex_study', 'n_maps': 2})
SCORE: 67.12471 at epoch 39

##################
###  Fold 003  ###
##################

1 trials detected for ('arc_type', 'lr', 'batch_size', 'epochs', 'cat_input_type', 'n_maps')

***** (1/1) *****
Search({'arc_type': 4, 'lr': 0.01, 'batch_size': 12, 'epochs': 40, 'cat_input_type': 'sex_study', 'n_maps': 2})
SCORE: 38.61472 at epoch 18

##################
###  Fold 00

<kerashypetune.KerasRandomSearchCV>

In [ ]:
pd.DataFrame(krs.best_params)

,arc_type,lr,batch_size,epochs,cat_input_type,steps_per_epoch
0,1,0.00005,4,17,sex_study,43
1,1,0.00005,4,11,sex_study,43
2,1,0.00005,4,19,sex_study,43
3,1,0.00005,4,19,sex_study,43


In [ ]:
pd.DataFrame(krs.folds_best_params)

,fold 1,fold 2,fold 3,fold 4
arc_type,1,1,1,1
lr,5e-05,5e-05,5e-05,5e-05
batch_size,4,4,4,4
epochs,17,11,19,19
cat_input_type,sex_study,sex_study,sex_study,sex_study
steps_per_epoch,43,43,43,43


In [ ]:
pd.DataFrame(krs.folds_scores)

,fold 1,fold 2,fold 3,fold 4
0,69.89584,28.81038,60.09507,64.32593
